In [20]:
import csv
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

In [21]:
from selenium import webdriver

In [22]:
driver = webdriver.Chrome()

In [23]:
url = 'https://amazon.de'
driver.get(url)

In [24]:
def get_url(search_term):
    template = 'https://www.amazon.de/s?k={}&language=de_DE&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)

In [25]:
print(get_url('möbelzubehör metall'))

https://www.amazon.de/s?k=möbelzubehör+metall&language=de_DE&ref=nb_sb_noss_1


## Creating soup

In [26]:
url = get_url('möbelzubehör metall')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type': 's-search-result'})
print(results[0])

<div class="sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20" data-asin="B09HH4DWL1" data-component-id="24" data-component-type="s-search-result" data-index="1" data-uuid="fcc10c71-80e1-4f99-82a8-71cf168a557d"><div class="sg-col-inner">
<div cel_widget_id="MAIN-SEARCH_RESULTS-1" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1">
<div class="rush-component s-expand-height" data-component-id="25" data-component-props='{"percentageShownToFire":"50","batchable":true,"requiredElementSelector":".s-image:visible","url":"https://unagi-eu.amazon.com/1/events/com.amazon.eel.SponsoredProductsEventTracking.prod?qualifier=1646240501&amp;id=3176002533127445&amp;widgetName=sp_atf&amp;adId=20090906929404&amp;eventType=1&amp;adIndex=0"}' data-component-type="s-impression-logger">
<div class="rush-component s-featured-result-item s-expand-height" data-com

## Getting data

In [8]:
item = results[0]
atag = item.h2.a
description = atag.text.strip()
print(description)

4 Stück Möbelfüße, Möbelfüsse aus Edelstahl 304, Für Schrankfüße Bett Beine/Sofa Beine/Tischbeine, Verwenden Sie M8-Schrauben, Unterstützung über 600 kg. (7.8cm)


In [9]:
url = 'https://amazon.de' + atag.get('href')

In [10]:
price_parent = item.find('span', 'a-price')
print(price_parent)

<span class="a-price" data-a-color="base" data-a-size="l"><span class="a-offscreen">29,99 €</span><span aria-hidden="true"><span class="a-price-whole">29,99</span><span class="a-price-symbol">€</span></span></span>


In [11]:
price = price_parent.find('span','a-offscreen').text
price = price.replace('€','')
price = price.replace(',','.')
price = price.strip(' ')
print(price)

29.99 


In [12]:
rating = item.i.text
rating = rating.replace(' von 5 Sternen', '')
rating = rating.replace(',', '.')
print(rating)

4.6


In [13]:
review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
print(review_count)

50


## Generalize the pattern

In [19]:
def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    to_translate = description
    translation = GoogleTranslator(source='de', target='pl').translate(to_translate)
    url = 'https://amazon.de' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
        price = price.replace('€','')
        price = price.replace(',','.')
        price = price.strip(' ')
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        rating = rating.replace(' von 5 Sternen', '')
        rating = rating.replace(',', '.')
        review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating=''
        review_count=''
    
    result = (description, translation, price, rating, review_count, url)
    
    return result

In [15]:
records = []

for item in results:
    record = (extract_record(item))
    if record:
        records.append(record)
    
print(records)

[('4 Stück Möbelfüße, Möbelfüsse aus Edelstahl 304, Für Schrankfüße Bett Beine/Sofa Beine/Tischbeine, Verwenden Sie M8-Schrauben, Unterstützung über 600 kg. (7.8cm)', '4-częściowe nóżki meblowe, 304 nóżki meblowe ze stali nierdzewnej, do nóżek szafki nogi łóżka/nogi sofy/nogi stołu, użyj śrub M8, wsparcie ponad 600 kg. (7,8cm)', '29.99\xa0', '4.6', '50', 'https://amazon.de/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02632421WPOZIXV36LYZ&url=%2FCXPDC-M%25C3%25B6belf%25C3%25BC%25C3%259Fe-M%25C3%25B6belbeine-h%25C3%25B6henverstellbare-Bolzenunterst%25C3%25BCtzungskraft%2Fdp%2FB07V4Y3T1C%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dm%25C3%25B6belzubeh%25C3%25B6r%2Bmetall%26qid%3D1646240355%26sr%3D8-1-spons%26psc%3D1&qualifier=1646240355&id=2154051067851759&widgetName=sp_atf'), ('4 Stück Ersatz Möbelfüße Einstellbare Aluminiumlegierung 10cm Metallfüße Schwarz Schrankfüsse Möbelfüsse, Metallfuß für Bett Sofa Füße und Tischbeine Schrankfüße Tischbeine. (10cm, Black)', '4 s

In [16]:
records[0]

('4 Stück Möbelfüße, Möbelfüsse aus Edelstahl 304, Für Schrankfüße Bett Beine/Sofa Beine/Tischbeine, Verwenden Sie M8-Schrauben, Unterstützung über 600 kg. (7.8cm)',
 '4-częściowe nóżki meblowe, 304 nóżki meblowe ze stali nierdzewnej, do nóżek szafki nogi łóżka/nogi sofy/nogi stołu, użyj śrub M8, wsparcie ponad 600 kg. (7,8cm)',
 '29.99\xa0',
 '4.6',
 '50',
 'https://amazon.de/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A02632421WPOZIXV36LYZ&url=%2FCXPDC-M%25C3%25B6belf%25C3%25BC%25C3%259Fe-M%25C3%25B6belbeine-h%25C3%25B6henverstellbare-Bolzenunterst%25C3%25BCtzungskraft%2Fdp%2FB07V4Y3T1C%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dm%25C3%25B6belzubeh%25C3%25B6r%2Bmetall%26qid%3D1646240355%26sr%3D8-1-spons%26psc%3D1&qualifier=1646240355&id=2154051067851759&widgetName=sp_atf')

In [17]:
for row in records:
    print(row[1])

4-częściowe nóżki meblowe, 304 nóżki meblowe ze stali nierdzewnej, do nóżek szafki nogi łóżka/nogi sofy/nogi stołu, użyj śrub M8, wsparcie ponad 600 kg. (7,8cm)
4 szt. Wymienne nóżki meblowe Regulowane stopy aluminiowe 10 cm metalowe nóżki czarne nóżki do szafek nóżki do mebli, metalowe nóżki do łóżka nóżki sofy i nogi do stołu nóżki do szafki nogi do stołu. (10cm, czarny)
Drenky 4-pak skośne nogi stołowe, metalowe nogi szafki, stożkowe nogi meblowe, stal nierdzewna 201, złota, wysokość 40 cm, antypoślizgowa cicha podstawa do sof, stołów i innych nóg meblowych
ToPicks nóżki meblowe regulowane metalowe, nogi stołowe ze stali nierdzewnej, nóżki meblowe z regulacją wysokości okrągłe stalowe nóżki rurowe sprężyste czarne, 6 cm 4 sztuki ze śrubami
DELSEN M8 regulowane nóżki do mebli nóżki poziomujące z nakrętkami wbijanymi metalowe prowadnice meblowe ze stali nierdzewnej do mebli sofa stół nogi do krzeseł
bieżniki Relaxdays, 2 sztuki, szer. x wys.: 40x43 cm, metalowe nogi stołu, biurko, stó

In [18]:
def get_url(search_term):
    template = 'https://www.amazon.de/s?k={}&language=de_DE&ref=nb_sb_noss_1'
    search_term = search_term.replace(' ','+')
    return template.format(search_term)

    url+='&page{}'

    return url

def extract_record(item):
    atag = item.h2.a
    description = atag.text.strip()
    to_translate = description
    translation = GoogleTranslator(source='de', target='pl').translate(to_translate)
    url = 'https://amazon.de' + atag.get('href')
    
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span','a-offscreen').text
        price = price.replace('€','')
        price = price.replace(',','.')
        price = price.strip(' ')
    except AttributeError:
        return
    
    try:
        rating = item.i.text
        rating = rating.replace(' von 5 Sternen', '')
        rating = rating.replace(',', '.')
        review_count = item.find('span',{'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating=''
        review_count=''
    
    result = (description, translation, price, rating, review_count, url)
    
    return result

def main(search_term):
    record = []
    url = get_url(search_term)
    
    for page in range(1, 7):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = (extract_record(item))
            if record:
                records.append(record)
            
    driver.close()
    
    with open('results.csv', 'w', newline = '', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title','Translation','Price','Rating','ReviewCount','Url'])
        writer.writerows(records)

In [109]:
main('möbelzubehör metall')

In [110]:
import pandas as pd

df = pd.read_csv('results.csv')

In [111]:
print(df.head())

                                               Title  \
0  2 x Bankgestell X-Gestell - Stahl Optik X8080 ...   
1  SHEENO 4 Stück Metalldreieck Möbel Füße,4 inch...   
2  4 Schwarz Metall Tischbeine DIY, Ersatz Möbelf...   
3  4 Stück Ersatz Möbelfüße Metall, 6" / 15cm Sch...   
4  DELSEN M8 Verstellbare füße für Möbel Nivellie...   

                                         Translation   Price  Rating  \
0   2 x rama ławki X rama - wygląd stali X8080 metal  99.99      4.2   
1  SHEENO 4 szt. Metalowe trójkątne nóżki meblowe...  15.99      1.0   
2  4 czarne metalowe nogi stołowe DIY wymienne nó...  41.99      NaN   
3  4 szt. Zamienne metalowe nóżki meblowe, 6"/15 ...  26.99      4.7   
4  DELSEN M8 regulowane nóżki do mebli nóżki pozi...  13.99      4.1   

   ReviewCount                                                Url  
0         27.0  https://amazon.de/gp/slredirect/picassoRedirec...  
1          1.0  https://amazon.de/gp/slredirect/picassoRedirec...  
2          NaN  https://am